In [ ]:
!pip install PyPDF2

In [ ]:
!pip install dateparser

In [4]:
import pandas as pd 
import numpy as np 
from PyPDF2 import PdfReader
from datetime import datetime 
import dateparser
import re
import locale
import dateparser
import json
import os

# Extracting the text 

In [13]:
def extract_pdf_to_txt(src_dir, dest_dir):
    # List all files in the source directory
    all_files = os.listdir(src_dir)

    # Filter PDF files
    pdf_files = [f for f in all_files if f.lower().endswith('.pdf')]

    # Loop through all the PDF files
    for pdf_file in pdf_files:
        pdf_path = os.path.join(src_dir, pdf_file)
        reader = PdfReader(pdf_path)

        # Extract text from the first page
        first_page_text = reader.pages[0].extract_text()

        # Search for a date in the format "DD MONTH YYYY" using a regular expression
        date_match = re.search(r'\d{1,2} (JANVIER|FÉVRIER|MARS|AVRIL|MAI|JUIN|JUILLET|AOÛT|SEPTEMBRE|OCTOBRE|NOVEMBRE|DÉCEMBRE)  \d{4}', first_page_text)

        # If a date is found, use it as a filename
        if date_match:
            # Get the matched date string
            date_str = date_match.group()

            # Remove extra spaces from the date string
            date_str = ' '.join(date_str.split())

            # Convert the date string to a datetime object
            date = dateparser.parse(date_str, languages=['fr'])

            # Format the date as "YY-MM-DD"
            formatted_date = date.strftime('%y-%m-%d')

            # Create a filename using the formatted date
            filename = f"{formatted_date}.txt"
        # If a date is not found, use the original PDF filename as a base
        else:
            filename = f"{os.path.splitext(pdf_file)[0]}.txt"

        txt_path = os.path.join(dest_dir, filename)

        # Open the output file for writing with 'utf-8' encoding
        with open(txt_path, "w", encoding='utf-8') as f:
            # Loop through all the pages in the PDF
            for page in reader.pages:
                # Extract text from the current page and write it to the output file
                f.write(page.extract_text())

src_dir = r'C:\Users\Stephanie\Documents\GitHub\NLP_Parlement\pdf'
dest_dir = r'C:\Users\Stephanie\Documents\GitHub\NLP_Parlement\txt'
extract_pdf_to_txt(src_dir, dest_dir)

# Important information

In [22]:
deputes = pd.read_csv(r'C:\Users\Stephanie\Documents\GitHub\NLP_Parlement\2022.07.22_Parlementaires PFWB_0.csv', header=None)
column_names = ['Prénom', 'nom', 'sexe', 'résidence', 'naissance', 'date', 'parti', 'titre']
deputes.columns = column_names

In [23]:
deputes

,Prénom,nom,sexe,résidence,naissance,date,parti,titre
0,Caroline,Cassart-Mailleux,F,Huy-Waremme,Namur,01/06/1973,MR,NaN
1,Philippe,Dodrimont,M,Liège,Aywaille,01/06/1964,MR,NaN
2,Jean-Luc,Crucke,M,Tournai-Ath-Mouscron,Renaix,29/10/1962,MR,NaN
3,Jean-Paul,Wahl,M,Nivelles,Uccle,17/11/1955,MR,NaN
4,Sybille,de Coster-Bauchau,F,Nivelles,Elisabethville,23/02/1953,MR,NaN
...,...,...,...,...,...,...,...,...
94,Pierre-Yves,Jeholet,M,NaN,Verviers,06/10/1968,MR,Ministre-Président
95,Bénédicte,Linard,F,NaN,Ottignies,22/08/1976,ECOLO,"Vice-Présidente et Ministre de l’Enfance, de l..."
96,Frédéric,Daerden,M,NaN,Montegnée,11/01/1970,PS,"Vice-Président et Ministre du Budget, de la Fo..."
97,Valérie,Glatigny,F,Auderghem,Marche-en-Famenne,13/11/1973,MR,"Ministre de l’Enseignement supérieur, de l’Ens..."


# Convert to JSON file

In [24]:
# Read the transcript text 
with open(r'txt\20-05-13.txt', 'r', encoding='utf-8') as f:
    full_text = f.read()

In [25]:
# Define regular expressions and helper function to parse questions, speakers and statements
def find_starting_point(text, starting_sentence='La séance est ouverte'):
    starting_point = text.find(starting_sentence)
    if starting_point != -1:
        return starting_point + len(starting_sentence)
    return None

# Start the text after the table des matieres 
starting_point = find_starting_point(full_text)
if starting_point is not None:
    transcript_text = full_text[starting_point:]
else:
    print("Starting sentence not found. Analyzing the full text.")
    transcript_text = full_text

In [26]:
# Clean the text from the \n characters 
transcript_text = transcript_text.replace('\n', ' ')
# Clean the text from the line breaks 
def clean_line_breaks(text):
    # Replace hyphenated line breaks with an empty string
    cleaned_text = re.sub(r'-\s+', '', text)
    return cleaned_text

transcript_text = clean_line_breaks(transcript_text)

In [27]:
# Helper
def find_questions(text):
    question_pattern = re.compile(r'(?<=\d\.\d Question)(.*?)(?=\d\.\d Question)', re.DOTALL)
    return question_pattern.findall(text)

def find_projets(text):
    projet_pattern = re.compile(r'\d+ Projet de décret(.*?)(?=\d+ Projet de décret|$)', re.DOTALL)
    return projet_pattern.findall(text)

def process_transcript(text, known_speakers):
    # Create a regex pattern for speaker names
    speaker_pattern = r'\b(?:' + '|'.join([re.escape(speaker["name"]) for speaker in known_speakers]) + r')\b'

    # Split the text into parts
    parts = re.split('(' + speaker_pattern + ')', text)
    # Initialize the result list
    result = []

    # Initialize the current speaker
    current_speaker = None

    # Iterate through the parts
    for part in parts:
        found_speaker = False
        for speaker in known_speakers:
            if speaker["name"] == part.strip():
                current_speaker = speaker
                found_speaker = True
                break

        if not found_speaker and current_speaker is not None:
            # Append the spoken text by the current speaker
            spoken_text = part.strip()
            if spoken_text:
                result.append({"speaker": current_speaker["name"], "title": current_speaker["title"], "text": spoken_text})

    return result


In [28]:
questions = find_questions(transcript_text)

In [31]:
questions[0]

' de M.  Pierre -Yves Lux à  M. Frédéric Daerden, vice -président  du gouvernement et ministre du Budget, de la Fo nction publique, de  l’Égalité des chances et de la tutelle  sur Wallonie -Bruxelles Enseignement,  intitulée «Réponse du ministre du  Budget à l’urgence culturelle»   M. Pierre -Yves Lux (Ecolo) . – Monsieur le  Ministre, votre intervention dans « La Libre » CRI No17 (2019 -2020)  ( 6 )  d’hier a  retenu toute mon attention pour trois raisons. Tout d’abord, vous y soulignez le rôle central que vous souhaitez donner ou redonner au  secteur culturel au sein de notre société. Vous  appelez à une discussion en profondeur sur cet  enjeu pour y parvenir. Le  groupe  Ecolo est particulièrement sensible à cette question.   Ensuite, vous appelez à une union sacrée afin  de répondre à l’urgence de la situation de crise  que traversent actuellement le secteur culturel et  tous ses opérateurs, comédiens  et techniciens ;  vous demandez à l’État fédéral de prendre ses  responsabilités

In [30]:
decrets = find_projets(transcript_text)

In [ ]:
result = process_transcript(questions[1], known_speakers)

In [ ]:
result = process_transcript(decrets[1], known_speakers)

In [ ]:
result

[{'speaker': 'Hélène Ryckmans', 'title': 'Ecolo', 'text': 'à M.'},
 {'speaker': 'Pierre -Yves Jeholet',
  'title': 'Ministre-Président',
  'text': ', ministre -président, intitulée «Intelligence artificielle  et risques»   Mme'},
 {'speaker': 'Hélène Ryckmans',
  'title': 'Ecolo',
  'text': '(Ecolo) . – Comme nous tous, j ’ai été sidérée d ’apprendre,  voici quelques jours, le décès d ’un jeune père de famille qui avait dialogué, au  sujet de son anxiété, avec le tchatb ot Eliza. Ce tchatbot l ’a entendu, l ’a écouté et  l’a poussé ou aidé à commettre son suicide. Nos pensées vont d ’abord à sa famille,  endeuillée. Ensuite, nous pensons à notre responsabilité, en tant que politiques,  d’agir face au phénomène.   La question de l ’intelligence artificielle a déjà été débattue ici, dans différentes  compétences de la Fédération Wallonie -Bruxelles: la jeunesse, l ’enseignement,  l’enseignement supérieur, l ’éducation, la culture. Nous devons définir un cadre à  l’intelligence artificiel

In [ ]:
# Create the json file 
def create_json_file(questions_data, projets_data, filename):
    data = {
        "questions": questions_data,
        "projets": projets_data,
    }
    with open(filename, "w") as outfile:
        json.dump(data, outfile, ensure_ascii=False, indent=2)

date = formatted_date
questions_type = "question"
projets_type = "projet"
theme = "to be defined later"
filename = f"{formatted_date}.json"

# Find and process questions
questions_text = find_questions(transcript_text)
questions_data = []
for question_text in questions_text:
    result = process_transcript(question_text, known_speakers)
    question_data = {
        "date": date,
        "type": questions_type,
        "theme": theme,
        "text": result
    }
    questions_data.append(question_data)

# Find and process projets
projets_text = find_projets(transcript_text)
projets_data = []
for projet_text in projets_text:
    result = process_transcript(projet_text, known_speakers)
    projet_data = {
        "date": date,
        "type": projets_type,
        "theme": theme,
        "text": result
    }
    projets_data.append(projet_data)

create_json_file(questions_data, projets_data, filename)